In [101]:
import xlwings as xw
from shapely.wkt import loads
import pandas as pd
import ezdxf
from ezdxf.addons import Importer
from ezdxf.math import Vec2

In [115]:
sheet = xw.sheets.active
zone_df = sheet.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
zone_df['Позиция номера'] = zone_df['Позиция номера'].apply(loads)
zone_df = zone_df[['Номер', 'Позиция номера']]

zone_df

,Номер,Позиция номера
0,208.0,POINT (3108.838205298518 1707.285109871409)
1,207.0,POINT (3114.150988844972 1707.4695572873902)
2,204.0,POINT (3115.690642463274 1707.5867820573599)
3,205.0,POINT (3115.1004453425708 1706.18062074892)
4,206.0,POINT (3114.699609216242 1705.0905155704238)
5,209.0,POINT (3108.9551269498993 1702.8311075061063)
6,210.0,POINT (3108.9551269498993 1702.8311075061063)
7,211.0,POINT (3104.0287726776705 1706.0783789373688)
8,217.0,POINT (3097.720077396069 1708.4442524397011)
9,200.0,POINT (3115.8767179014208 1710.2258575299381)


In [126]:
zip_numbers = {}
for _, series in zone_df.iterrows():
    if series['Позиция номера'] not in zip_numbers:
        zip_numbers[series['Позиция номера']] = [series['Номер']]
    else:
        zip_numbers[series['Позиция номера']].append(series['Номер'])

zip_numbers

{<POINT (3108.838 1707.285)>: [208.0],
 <POINT (3114.151 1707.47)>: [207.0],
 <POINT (3115.691 1707.587)>: [204.0],
 <POINT (3115.1 1706.181)>: [205.0],
 <POINT (3114.7 1705.091)>: [206.0],
 <POINT (3108.955 1702.831)>: [209.0, 210.0],
 <POINT (3104.029 1706.078)>: [211.0],
 <POINT (3097.72 1708.444)>: [217.0],
 <POINT (3115.877 1710.226)>: [200.0, 201.0, 202.0, 203.0],
 <POINT (3102.169 1708.507)>: [215.0, 216.0],
 <POINT (3102.448 1705.577)>: [212.0, 213.0, 214.0]}

In [130]:
def compress_ranges(numbers):
    numbers = sorted(map(int, numbers))
    result = []
    start = numbers[0]
    end = start

    for i in range(1, len(numbers)):
        if numbers[i] == end + 1:
            end = numbers[i]
        else:
            if end - start == 1:
                result.append(f"{start},{end}")
            else:
                result.append(f"{start}-{end}" if start != end else str(start))
            start = end = numbers[i]

    if end - start == 1:
        result.append(f"{start},{end}")
    else:
        result.append(f"{start}-{end}" if start != end else str(start))
    
    return ",".join(result)

compress_ranges(['200', '201', '202', '204', '205', '209', '210', '211', '213'])

'200-202,204,205,209-211,213'

In [137]:
zip_numbers_text = {k: compress_ranges(v) for k, v in zip_numbers.items()}

zip_numbers_text

{<POINT (3108.838 1707.285)>: '208',
 <POINT (3114.151 1707.47)>: '207',
 <POINT (3115.691 1707.587)>: '204',
 <POINT (3115.1 1706.181)>: '205',
 <POINT (3114.7 1705.091)>: '206',
 <POINT (3108.955 1702.831)>: '209,210',
 <POINT (3104.029 1706.078)>: '211',
 <POINT (3097.72 1708.444)>: '217',
 <POINT (3115.877 1710.226)>: '200-203',
 <POINT (3102.169 1708.507)>: '215,216',
 <POINT (3102.448 1705.577)>: '212-214'}

In [141]:
zip_numbers_text_for_df = [{'Номер': v, 'Позиция номера': k} for k, v in zip_numbers_text.items()]

zip_numbers_text_for_df

[{'Номер': '208', 'Позиция номера': <POINT (3108.838 1707.285)>},
 {'Номер': '207', 'Позиция номера': <POINT (3114.151 1707.47)>},
 {'Номер': '204', 'Позиция номера': <POINT (3115.691 1707.587)>},
 {'Номер': '205', 'Позиция номера': <POINT (3115.1 1706.181)>},
 {'Номер': '206', 'Позиция номера': <POINT (3114.7 1705.091)>},
 {'Номер': '209,210', 'Позиция номера': <POINT (3108.955 1702.831)>},
 {'Номер': '211', 'Позиция номера': <POINT (3104.029 1706.078)>},
 {'Номер': '217', 'Позиция номера': <POINT (3097.72 1708.444)>},
 {'Номер': '200-203', 'Позиция номера': <POINT (3115.877 1710.226)>},
 {'Номер': '215,216', 'Позиция номера': <POINT (3102.169 1708.507)>},
 {'Номер': '212-214', 'Позиция номера': <POINT (3102.448 1705.577)>}]

In [142]:
zip_numbers_text_df = pd.DataFrame(zip_numbers_text_for_df)

zip_numbers_text_df

,Номер,Позиция номера
0,208,POINT (3108.838205298518 1707.285109871409)
1,207,POINT (3114.150988844972 1707.4695572873902)
2,204,POINT (3115.690642463274 1707.5867820573599)
3,205,POINT (3115.1004453425708 1706.18062074892)
4,206,POINT (3114.699609216242 1705.0905155704238)
5,"209,210",POINT (3108.9551269498993 1702.8311075061063)
6,211,POINT (3104.0287726776705 1706.0783789373688)
7,217,POINT (3097.720077396069 1708.4442524397011)
8,200-203,POINT (3115.8767179014208 1710.2258575299381)
9,"215,216",POINT (3102.168668723462 1708.5065675452358)


In [151]:
# Имитация определения действия

from random import randint

actions = []
for i in range(11):
    actions.append('Удаление' if randint(0, 1) == 0 else 'Пересадка')

zip_numbers_text_df['Действие'] = actions

zip_numbers_text_df

,Номер,Позиция номера,Действие
0,208,POINT (3108.838205298518 1707.285109871409),Удаление
1,207,POINT (3114.150988844972 1707.4695572873902),Удаление
2,204,POINT (3115.690642463274 1707.5867820573599),Удаление
3,205,POINT (3115.1004453425708 1706.18062074892),Удаление
4,206,POINT (3114.699609216242 1705.0905155704238),Пересадка
5,"209,210",POINT (3108.9551269498993 1702.8311075061063),Удаление
6,211,POINT (3104.0287726776705 1706.0783789373688),Удаление
7,217,POINT (3097.720077396069 1708.4442524397011),Удаление
8,200-203,POINT (3115.8767179014208 1710.2258575299381),Удаление
9,"215,216",POINT (3102.168668723462 1708.5065675452358),Удаление


In [179]:
import ezdxf
from ezdxf.addons import Importer
from ezdxf.math import Vec2
from datetime import datetime

In [170]:
dxf_template_path = r'C:\Projects\TaxationTool\data\template.dxf'
dt = datetime.now().strftime('%Y%m%d%H%M%S')
dxf_output_path = fr'C:\Projects\TaxationTool\test_data\dxf_out_{dt}.dxf'
data = zip_numbers_text_df

In [171]:
layer_name_taxation_removable = 'Таксация_деревья(удаляемые)'
layer_name_taxation_transplantable = 'Таксация_деревья(пересаживаемые)'
layer_name_leader_removable = 'Таксация_номера(удаляемые)'
layer_name_leader_transplantable = 'Таксация_номера(пересаживаемые)'
block_scale = 0.004
dxfattribs_block = {'xscale': block_scale,
                    'yscale': block_scale}
mtext_segment = Vec2.from_deg_angle(45, 3)
block_name_removable = 'taxation_removable'
block_name_transplantable = 'taxation_removable'

In [180]:
dxf_source = ezdxf.readfile(dxf_template_path)
dxf_new = ezdxf.new(dxfversion='R2013', setup=True, units=6)
msp = dxf_new.modelspace()

importer = Importer(dxf_source, dxf_new)
importer.import_block(block_name_removable)
importer.import_block(block_name_transplantable)
importer.finalize()

# dxf_new.layers.add(layer_name_taxation_removable, color=7)
# dxf_new.layers.add(layer_name_taxation_transplantable, color=7)
# dxf_new.layers.add(layer_name_leader_removable, color=7)
# dxf_new.layers.add(layer_name_leader_transplantable, color=7)

In [181]:
for _, series in data.iterrows():
    insert_point = Vec2(series['Позиция номера'].x, series['Позиция номера'].y)
    text_number = series['Номер']
    
    action = series['Действие']
    if action == 'Удаление':
        dxfattribs_block['layer'] = layer_name_taxation_removable
        layer = layer_name_leader_removable
        block_name = block_name_removable
    elif action == 'Пересадка':
        dxfattribs_block['layer'] = layer_name_taxation_transplantable
        layer = layer_name_leader_transplantable
        block_name = block_name_transplantable
    else:
        raise ValueError(f"Номер {text_number} - неизвестное действие '{action}'")
    
    msp.add_blockref(block_name, insert_point, dxfattribs=dxfattribs_block)
    ml_builder = msp.add_multileader_mtext(style='Standard', dxfattribs={'layer': layer})
    ml_builder.quick_leader(text_number, insert_point, mtext_segment)

dxf_new.saveas(dxf_output_path)